<a href="https://colab.research.google.com/github/Super-rookie-Py/_WebProgramming_Class/blob/master/_%EB%B0%95%EA%B1%B4%EC%9A%B0%5B%EA%B3%BC%EC%A0%9C%5D_Selenium%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EB%8F%99%EC%A0%81_%EC%9B%B9_%ED%8E%98%EC%9D%B4%EC%A7%80_%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 네이버웹툰 추천 목록에서 베스트댓글 스크래핑
- https://comic.naver.com/index.nhn 네이버 웹툰
- keonwoo Park 2020/09/17

In [1]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

     |████████████████████████████████| 911kB 4.1MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.

In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

## webtoon_scraping( ): 해당 웹툰 스크래핑

In [3]:
def webtoon_scraping(title_url, wd):
  
  print("[ 웹툰 스크래핑 시작 ]")
  comic_info = wd.find_element_by_class_name("comicinfo")
  author = comic_info.find_element_by_class_name('wrt_nm').text
  title = comic_info.find_element_by_css_selector('#content > div.comicinfo > div.detail > h2').text[:-len(author)]
  comic_intro = comic_info.find_element_by_tag_name('p').text
  comic_recomm = comic_info.find_element_by_class_name('u_cnt').text
  print("제목:", title,"작가:",author,"추천:", comic_recomm )

  comic_view = wd.find_element_by_class_name('viewList')
  comic_list = comic_view.find_elements_by_class_name('title')

  print("[ 웹툰 스크래핑 끝 ]")
  return comic_list, [title, author, comic_recomm]

## best_comment( ): 베스트 댓글 스크래핑

In [22]:
def best_comment(comic_url, wd):

  comment_df = pd.DataFrame(columns=("Nick_name", "Comment", "Recommend", "UnRecommend"))
  idx = 0
  print(["####### 댓글 스크래핑 시작 #######"])
  comment_box = wd.find_element_by_class_name('u_cbox_content_wrap')
  best_comments = comment_box.find_elements_by_class_name('u_cbox_comment_box')
  
  for best_comment in best_comments:
    try:
      nick_name = best_comment.find_element_by_class_name('u_cbox_name_area').text
      comment = best_comment.find_element_by_class_name('u_cbox_contents').text
      recomm = best_comment.find_element_by_class_name('u_cbox_cnt_recomm').text
      unrecomm = best_comment.find_element_by_class_name('u_cbox_cnt_unrecomm').text
      comment_df.loc[idx] = [nick_name, comment, recomm, unrecomm]
      idx += 1
      print('#',idx, nick_name, comment, "좋아요:",recomm, "싫어요:",unrecomm)
    except NoSuchElementException as e:
      print(e)
      continue
    except StaleElementReferenceException as e:
      print(e)
      continue    
  print("####### 댓글 끝 #######")
  return comment_df

# scraping(max_count): 스크래핑 메인함수
- max_count : 선택된 웹툰 중 가장 최근에 올라온 웹툰의 갯수

In [32]:
def scraping(max_count = 1):
  
  comic_df = pd.DataFrame(columns=("Title","Author", "Recommend"))
  idx = 0
  wd = webdriver.Chrome('chromedriver',options=chrome_options)
  wd.implicitly_wait(3)
  wd.execute_script('window.open("about:blank", "blank");')
  wd.execute_script('window.open("about:blank", "_blank");')
  wd.execute_script('window.open("about:blank", "_blank");')
 
  tabs = wd.window_handles

  wd.switch_to.window(tabs[0])
  main_url = 'https://comic.naver.com/index.nhn'
  wd.get(main_url)
  comment_df = pd.DataFrame()
  genre_ul = wd.find_element_by_class_name('tab_gr')
  genre_list = genre_ul.find_elements_by_tag_name('a')
  

  for genre in genre_list:
    genre.click()
    time.sleep(1)
    print("[", genre.text, "]")
    genre_rec_list = wd.find_elements_by_class_name('genreRecomInfo2')
    for genre_rec in genre_rec_list:
      title_class = genre_rec.find_element_by_class_name('title')
      title_url = title_class.find_element_by_tag_name('a').get_attribute("href")
      wd.switch_to.window(tabs[1])
      wd.get(title_url)
      print("title: ",title_url)

      # 웹툰 스크래핑
      comic_list, comic_df.loc[idx] = webtoon_scraping(title_url, wd)
      idx += 1
      count = 0
      for comic in comic_list:
        try:
          if count == max_count: break
          comic_url = comic.find_element_by_tag_name("a").get_attribute("href")
          print(comic_url)
          comic_title = comic.find_element_by_tag_name("a").text
          print("################", comic_title,"################")
          wd.switch_to.window(tabs[2])
          wd.implicitly_wait(3)
          wd.get(comic_url)
          iframe = wd.find_element_by_class_name('sub_cont')
          iframe_url = iframe.find_element_by_id("commentIframe").get_attribute("src")
          count += 1
        except NoSuchElementException as e:
          print(e)
          break


        wd.switch_to.window(tabs[3])
        print("tab 3")
        wd.implicitly_wait(3)
        wd.get(iframe_url)
        # 댓글 스크래핑
        try:
          df = best_comment(iframe_url, wd)
          comment_df = pd.concat([comment_df, df])
        except NoSuchElementException as e:
          print(e)
          continue
        except StaleElementReferenceException as e:
          print(e)

        wd.switch_to.window(tabs[2])
        wd.implicitly_wait(3)

        wd.switch_to.window(tabs[1])
        wd.implicitly_wait(3)

      wd.switch_to.window(tabs[0])
      wd.implicitly_wait(3)
    
      
  
  return comic_df, comment_df

In [33]:
comic_df, comment_df = scraping()

[ 에피소드 ]
title:  https://comic.naver.com/webtoon/list.nhn?titleId=711422
[ 웹툰 스크래핑 시작 ]
제목: 삼국지톡컷툰  작가: 무적핑크 / 이리 추천: 99,999+
[ 웹툰 스크래핑 끝 ]
https://comic.naver.com/webtoon/detail.nhn?titleId=711422&no=238&weekday=tue
################ 관도대전_53.처형대에 선 여포 ################
tab 3
['####### 댓글 스크래핑 시작 #######']
# 1 오영석
(aya2****) 저때 살려줬으면 아빠로만 쿼드라킬 했을 여포 당신은 도대체..... 좋아요: 3618 싫어요: 21
# 2 Winter02
(hmr1****) 뒤틀린 황천의 유비 좋아요: 2729 싫어요: 17
# 3 망탕파 불주먹
(hong****) 말년에도 술주정 안했습니다 ... 기록에 술이 언급한 적이 아예 없습니다... 부하한테 박한게 흠이었지....... 좋아요: 2441 싫어요: 22
# 4 KKXX
(dlqu****) 유비는 ㄹㅇ 현시대에서 주식했으면 워렌버핏쯤은 가볍게 눌러버렸을듯매도 타이밍이 기가맥힘 ㄹㅇ떡락하기 바로 직전에 귀신처럼 잘 놀다 갑니다~하고 나감 좋아요: 2226 싫어요: 16
# 5 김민기
(7118****) 진짜ㅠㅠ 나중에 미방이 관우 배신하고 오에 붙은거 소식듣고 미축이 스스로를 밧줄에 묶은뒤에 유비한테 찾아가서 죽여달라고 했지만 유비는 미축의 죄가 아니라고 감쌌지만 몇일뒤 병으로 죽었다는데... 난 못볼꺼 같아ㅠㅠ............... 좋아요: 2149 싫어요: 27
# 6 (qiqi****) 와 다음주면...여포 살려달라고 자기 쓸모 줄줄이 말하고 유비는 그때 쪼레기에게 "승상은 동탁과 정원의 일을 잊으셨습니까?"라고 막타 치는 거 볼 수 있는겨? 플러스 진궁이랑 조조는 드디어 찐 이별일세 ㅠㅜ 좋아요: 2128 싫어요: 24
# 7 Arastar
(kkkk*

In [34]:
comic_df

,Title,Author,Recommend
0,삼국지톡컷툰,무적핑크 / 이리,"99,999+"
1,화장 지워주는 남자,이연,"99,999+"
2,2020 최애캐의 MBTI컷툰,웹툰작가,"99,999+"
3,경기딸,이윤창,"23,522"
4,오늘의 순정망화,손하기,"99,999+"
...,...,...,...
73,불릿 6미리,김도근,"22,303"
74,격기3반,이학,"99,999+"
75,빌드업,911,"38,305"
76,윈드브레이커,조용석,"99,999+"


In [35]:
comment_df

,Nick_name,Comment,Recommend,UnRecommend
0,오영석\n(aya2****),저때 살려줬으면 아빠로만 쿼드라킬 했을 여포 당신은 도대체.....,3618,21
1,Winter02\n(hmr1****),뒤틀린 황천의 유비,2729,17
2,망탕파 불주먹\n(hong****),말년에도 술주정 안했습니다 ... 기록에 술이 언급한 적이 아예 없습니다... 부하...,2441,22
3,KKXX\n(dlqu****),유비는 ㄹㅇ 현시대에서 주식했으면 워렌버핏쯤은 가볍게 눌러버렸을듯매도 타이밍이 기가...,2226,16
4,김민기\n(7118****),진짜ㅠㅠ 나중에 미방이 관우 배신하고 오에 붙은거 소식듣고 미축이 스스로를 밧줄에 ...,2149,27
...,...,...,...,...
10,Qwerty0987\n(wnsg****),공태성이 한말중에 “대학가면 농구는 진짜 때려친다” 이런 말이 있음다르게 말하면“그...,1309,10
11,IlIllllIllI\n(shxo****),ㄹㅇ 과학2점짜리가 킬러문제였던거임 ㅋㅋ,919,13
12,(ro76****),엄마 나 농구..??: 어 거기 대치동 **학원 맞죠?,863,14
13,정대만은 정대만\n(rlae****),여기서 주목할 건 태성이 머리도 있지만 농구를 향한 열정이 매우 크다는 것,716,19
